In [86]:
import requests
import datetime
import base64
import shutil
from urllib.parse import urlencode

In [2]:
from config import client_id, client_secret

In [63]:
class SpotifyAPI(object):
    access_token = None
    access_token_expires = datetime.datetime.now()
    access_token_is_expired = True
    client_id = None
    client_secret = None
    token_url = "https://accounts.spotify.com/api/token"
    
    def __init__(self, client_id, client_secret):
        self.client_id = client_id
        self.client_secret = client_secret
    
    def get_client_credentials(self):
        """
        Returns a base64 encoded string
        """
        client_id = self.client_id
        client_secret = self.client_secret
        
        if client_id == None or client_secret == None:
            raise Exception("Requires client_id and client_secret")
            
        client_creds = f"{client_id}:{client_secret}"
        client_creds_b64 = base64.b64encode(client_creds.encode())
        return client_creds_b64.decode()
    
    def get_token_headers(self):
        client_creds_b64 = self.get_client_credentials()
        return {
            "Authorization": f"Basic {client_creds_b64}"
        }
    
    def get_token_data(self):
        return {
            "grant_type": "client_credentials"
        }
    
    def perform_auth(self):
        token_url = self.token_url
        token_data = self.get_token_data()
        token_headers = self.get_token_headers()
        r = requests.post(token_url, data=token_data, headers=token_headers)
        if r.status_code not in range(200, 299):
            raise Exception("Unable to authenticate client.")
        data = r.json()
        now = datetime.datetime.now()
        access_token = data['access_token']
        expires_in = data['expires_in'] # seconds
        expires = now + datetime.timedelta(seconds=expires_in)
        self.access_token = access_token
        self.access_token_expires = expires
        self.access_token_is_expired = expires < now
        return True
    
    def get_access_token(self):
        token = self.access_token
        expires = self.access_token_expires
        now = datetime.datetime.now()
        if expires < now:
            self.perform_auth()
            return self.get_access_token()
        elif token == None:
            self.perform_auth()
            return self.get_access_token() 
        return token
    
    def get_resource_header(self):
        access_token = self.get_access_token()
        headers = {
            "Authorization": f"Bearer {access_token}"
        }
        return headers
    
    def get_resource(self, lookup_id, resource_type='albums', version='v1'):
        endpoint = f"https://api.spotify.com/{version}/{resource_type}/{lookup_id}"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
    
    def get_album(self, lookup_id):
        return self.get_resource(lookup_id, resource_type='albums')
    
    def get_artist_albums(self, lookup_id, version='v1'):
        endpoint = f"https://api.spotify.com/{version}/artists/{lookup_id}/albums"
        headers = self.get_resource_header()
        r = requests.get(endpoint, headers=headers)
        if r.status_code not in range(200, 299):
            return {}
        return r.json()
        
    def search(self, query, search_type='artist' ): # type
        headers = self.get_resource_header()
        endpoint = "https://api.spotify.com/v1/search"
        data = urlencode({"q": query, "type": search_type.lower()})
        lookup_url = f"{endpoint}?{data}"
        r = requests.get(lookup_url, headers=headers)
        if r.status_code not in range(200, 299):  
            return {}
        return r.json() 

In [64]:
spotify = SpotifyAPI(client_id, client_secret)

In [51]:
spotify.search('Bon Jovi')

{'artists': {'href': 'https://api.spotify.com/v1/search?query=Bon+Jovi&type=artist&offset=0&limit=20',
  'items': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/58lV9VcRSjABbAbfWS6skp'},
    'followers': {'href': None, 'total': 8834708},
    'genres': ['glam metal', 'rock'],
    'href': 'https://api.spotify.com/v1/artists/58lV9VcRSjABbAbfWS6skp',
    'id': '58lV9VcRSjABbAbfWS6skp',
    'images': [{'height': 640,
      'url': 'https://i.scdn.co/image/dd5408b179b5792445e107c188a3c3a213a19f09',
      'width': 640},
     {'height': 320,
      'url': 'https://i.scdn.co/image/0f8139bf1b88a2c9ee861c28f957025c2570995a',
      'width': 320},
     {'height': 160,
      'url': 'https://i.scdn.co/image/eacfcd623c24a6f774cb8733cabf60b4e80ba031',
      'width': 160}],
    'name': 'Bon Jovi',
    'popularity': 81,
    'type': 'artist',
    'uri': 'spotify:artist:58lV9VcRSjABbAbfWS6skp'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/6h2bWHWTJL38N8dqocVaif'},
    '

In [60]:
spotify.get_resource('58lV9VcRSjABbAbfWS6skp', 'artists')

{'external_urls': {'spotify': 'https://open.spotify.com/artist/58lV9VcRSjABbAbfWS6skp'},
 'followers': {'href': None, 'total': 8834708},
 'genres': ['glam metal', 'rock'],
 'href': 'https://api.spotify.com/v1/artists/58lV9VcRSjABbAbfWS6skp',
 'id': '58lV9VcRSjABbAbfWS6skp',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/dd5408b179b5792445e107c188a3c3a213a19f09',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/0f8139bf1b88a2c9ee861c28f957025c2570995a',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/eacfcd623c24a6f774cb8733cabf60b4e80ba031',
   'width': 160}],
 'name': 'Bon Jovi',
 'popularity': 81,
 'type': 'artist',
 'uri': 'spotify:artist:58lV9VcRSjABbAbfWS6skp'}

In [59]:
spotify.get_album('72oWQmX0oUXXi5YzMtpwUv')

{}

In [89]:
for album in spotify.get_artist_albums('58lV9VcRSjABbAbfWS6skp')[ 'items']:
    print(f"{album['name']}")
    for image in album['images']:
         
        if image['height'] == 640:
            print(image['url'])
            image_url = image['url']
            filename = image_url.split("/")[-1]
            
            r = requests.get(image_url, stream = True)
            
            
            # Check if the image was retrieved successfully
            if r.status_code == 200:
                # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                r.raw.decode_content = True
    
                # Open a local file with wb ( write binary ) permission.
                with open(filename+'.jpg','wb') as f:
                    shutil.copyfileobj(r.raw, f)
        
                print('Image sucessfully Downloaded: ',filename)
            else:
                print('Image Couldn\'t be retreived')

2020 (Deluxe)
https://i.scdn.co/image/ab67616d0000b2739f7ea39890e50740cd33be09
Image sucessfully Downloaded:  ab67616d0000b2739f7ea39890e50740cd33be09
2020
https://i.scdn.co/image/ab67616d0000b2738e60cfe087f24efa28e868cc
Image sucessfully Downloaded:  ab67616d0000b2738e60cfe087f24efa28e868cc
This House Is Not For Sale
https://i.scdn.co/image/ab67616d0000b273ab556d12ef113989ed5bf2e3
Image sucessfully Downloaded:  ab67616d0000b273ab556d12ef113989ed5bf2e3
This House Is Not For Sale (Deluxe)
https://i.scdn.co/image/ab67616d0000b2731c033232365bceea666f6041
Image sucessfully Downloaded:  ab67616d0000b2731c033232365bceea666f6041
This House Is Not For Sale (Deluxe)
https://i.scdn.co/image/ab67616d0000b273b6bae2935543289fde45e536
Image sucessfully Downloaded:  ab67616d0000b273b6bae2935543289fde45e536
This House Is Not For Sale (Live From The London Palladium)
https://i.scdn.co/image/ab67616d0000b273434059812d9bb02b6e28ca80
Image sucessfully Downloaded:  ab67616d0000b273434059812d9bb02b6e28ca80
